# SelfregulationSCP1

In [1]:
###libraries

#Data manipulation
import pandas as pd
import numpy as np
from collections import Counter
from sklearn import preprocessing
import sys
import os

#Time series transformers
from pyts.multivariate.classification import MultivariateClassifier
from pyts.multivariate.transformation import WEASELMUSE
from pyts.classification import KNeighborsClassifier
from pywt import wavedec
import pyeeg
import scipy.stats

#Classifiers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

#Deep Learning
import mcfly

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from keras import backend
from numba import cuda 

#Random
import multiprocessing
from multiprocessing import Pool
import gc

np.random.seed(42)

Using TensorFlow backend.


In [2]:
###Read the data
dataset = 'SelfregulationSCP1'

X_train = np.load('Datasets_clean/{}/X_train.npy'.format(dataset))
y_train = np.load('Datasets_clean/{}/y_train.npy'.format(dataset))
X_test = np.load('Datasets_clean/{}/X_test.npy'.format(dataset))
y_test = np.load('Datasets_clean/{}/y_test.npy'.format(dataset))

X_full = np.vstack([X_train,X_test])
y_full = np.hstack([y_train,y_test])

#Create scores dict
scores = {}

### Functions

In [3]:
def ResampleLinear1D(original, targetLen = 40):
    original = np.array(original, dtype=np.float)
    index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.float)
    index_floor = np.array(index_arr, dtype=np.int) #Round down
    index_ceil = index_floor + 1
    index_rem = index_arr - index_floor #Remain

    val1 = original[index_floor]
    val2 = original[index_ceil % len(original)]
    interp = val1 * (1.0-index_rem) + val2 * index_rem
    assert(len(interp) == targetLen)
    return interp

def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
 
def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

### DTW + 1-Knn

In [4]:
dtw_acc = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]
    
#     X_train = np.apply_along_axis(ResampleLinear1D, axis = 2, arr = X_train)
#     X_test = np.apply_along_axis(ResampleLinear1D, axis = 2, arr = X_test)
    
    dtw_knn = MultivariateClassifier(KNeighborsClassifier(metric = 'dtw_itakura',
                                                      n_jobs = -1,
                                                      metric_params = {'max_slope' : 2}))
    
    dtw_knn.fit(X_train,y_train)
    acc = dtw_knn.score(X_test,y_test)
    dtw_acc.append(acc)
    
scores['dtw_knn'] = [np.mean(dtw_acc), np.std(dtw_acc)]

### WEASELMUSE

In [5]:
accuracies_lr = []
accuracies_rf = []
accuracies_svc = []
accuracies_lgbm = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]

    #Optimal hyperparameters
    hyperparameters = {}

    #wm + lr    
    hyperparameters['wm_lr'] = {}
    hyperparameters['wm_lr']['word_size'] = 2
    hyperparameters['wm_lr']['n_bins'] = 3
    hyperparameters['wm_lr']['C'] = 0.5

    wm = WEASELMUSE(word_size = hyperparameters['wm_lr']['word_size'], 
                    n_bins = hyperparameters['wm_lr']['n_bins'])

    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['wm_lr']['C'])

    clf = make_pipeline(wm, logistic)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_lr.append(np.round(acc,2))
    
    #wm + rf    
    hyperparameters['wm_rf'] = {}
    hyperparameters['wm_rf']['word_size'] = 2
    hyperparameters['wm_rf']['n_bins'] = 3
    hyperparameters['wm_rf']['n_estimators'] = 300
    hyperparameters['wm_rf']['max_depth'] = 9

    wm = WEASELMUSE(word_size = hyperparameters['wm_rf']['word_size'], 
                    n_bins = hyperparameters['wm_rf']['n_bins'])

    rf = RandomForestClassifier(n_estimators = hyperparameters['wm_rf']['n_estimators'],
                                max_depth = hyperparameters['wm_rf']['max_depth'])

    clf = make_pipeline(wm, rf)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_rf.append(np.round(acc,2))

    #wm + svc
    hyperparameters['wm_svc'] = {}
    hyperparameters['wm_svc']['word_size'] = 2
    hyperparameters['wm_svc']['n_bins'] = 3
    hyperparameters['wm_svc']['C'] = 1
    hyperparameters['wm_svc']['kernel'] = 'rbf'
    hyperparameters['wm_svc']['degree'] = 3
    hyperparameters['wm_svc']['gamma'] = 'scale'

    wm = WEASELMUSE(word_size = hyperparameters['wm_svc']['word_size'], 
                    n_bins = hyperparameters['wm_svc']['n_bins'])

    svc = SVC(C = hyperparameters['wm_svc']['C'],
              kernel = hyperparameters['wm_svc']['kernel'],
              degree = hyperparameters['wm_svc']['degree'],
              gamma = hyperparameters['wm_svc']['gamma'])

    clf = make_pipeline(wm, svc)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_svc.append(np.round(acc,2))

    #wm + lgbm
    hyperparameters['wm_lgbm'] = {}
    hyperparameters['wm_lgbm']['word_size'] = 2
    hyperparameters['wm_lgbm']['n_bins'] = 3
    hyperparameters['wm_lgbm']['num_leaves'] = 254
    hyperparameters['wm_lgbm']['max_depth'] = 2
    hyperparameters['wm_lgbm']['learning_rate'] = 0.1
    hyperparameters['wm_lgbm']['n_estimators'] = 500
    hyperparameters['wm_lgbm']['min_split_gain'] = 0.2
    hyperparameters['wm_lgbm']['min_child_samples'] = 20
    hyperparameters['wm_lgbm']['colsample_bytree'] = 1
    hyperparameters['wm_lgbm']['reg_alpha'] = 0.1
    hyperparameters['wm_lgbm']['reg_lambda'] = 0

    wm = WEASELMUSE(word_size = hyperparameters['wm_svc']['word_size'], 
                    n_bins = hyperparameters['wm_svc']['n_bins'])

    def sparse_float(mat):
        return mat.astype('float')

    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['wm_lgbm']['num_leaves'],
                          max_depth = hyperparameters['wm_lgbm']['max_depth'],
                          learning_rate = hyperparameters['wm_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['wm_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['wm_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['wm_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['wm_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['wm_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['wm_lgbm']['reg_lambda'])

    clf = make_pipeline(wm, trans_sparse_float, lgbm)
    clf.fit(X_train,y_train)
    acc = clf.score(X_test,y_test)
    accuracies_lgbm.append(np.round(acc,2))
    
scores['wm_lr'] = [np.mean(accuracies_lr), np.std(accuracies_lr)]
scores['wm_rf'] = [np.mean(accuracies_rf), np.std(accuracies_rf)]
scores['wm_svc'] = [np.mean(accuracies_svc), np.std(accuracies_svc)]
scores['wm_lgbm'] = [np.mean(accuracies_lgbm), np.std(accuracies_lgbm)]

### Deep Learning

In [6]:
def dl_func(dl_type,
            X_train_dl, 
            y_train_dl,
            X_val_dl,
            y_val_dl,
            X_test_dl,
            y_test_dl):
    
    #Validate diferent architectures
    num_of_candidate_models = 8
    random_search_epoches = 100
    random_search_es = 30
    best_model_epoches = 200
    best_model_es = 30
    
    for mod_type in [dl_type]:

        #Create architectures
        num_classes = y_train_dl.shape[1]
        metric = 'accuracy'
        models = mcfly.modelgen.generate_models(X_train_dl.shape,
                                                number_of_classes=num_classes,
                                                number_of_models = num_of_candidate_models,
                                                model_types = [mod_type],
                                                metrics=[metric])

        #Save intermediate results
        resultpath = 'temp'
        outputfile = os.path.join(resultpath, 'modelcomparison_{}_{}.json'.format(mod_type,dataset))

        #Find best architecture
        histories, val_accuracies, val_losses = mcfly.find_architecture.train_models_on_samples(X_train_dl, y_train_dl,
                                                                                                X_val_dl, y_val_dl,
                                                                                                models,
                                                                                                nr_epochs=random_search_epoches,
                                                                                                subset_size=None,
                                                                                                verbose=False,
                                                                                                outputfile=outputfile,
                                                                                                metric=metric,
                                                                                                early_stopping_patience=random_search_es
                                                                                                )

        #Select and train best architecture
        best_model_index = np.argmax(val_accuracies)
        best_model, best_params, best_model_types = models[best_model_index]

        es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=best_model_es)
        mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

        history = best_model.fit(X_train_dl, y_train_dl,
                                 epochs=best_model_epoches, validation_data=(X_val_dl, y_val_dl),
                                 callbacks = [es,mc])

        #Accuracy in test dataset
        saved_model = load_model('best_model.h5')
        return saved_model.evaluate(X_test_dl, y_test_dl, verbose = False)[1]

In [7]:
accuracies_cnn = []
accuracies_it = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]

    #Create train, validation and test sets for DL
    LE = LabelEncoder()
    skf_train_val = StratifiedKFold(n_splits=5)

    for train_index, val_index in skf_train_val.split(X_train, y_train):
        train_index = train_index
        val_index = val_index

    X_train_val = X_train.copy()

    X_train_dl = X_train_val[train_index].copy()
    X_train_dl = np.transpose(X_train_dl, (0,2,1))
    y_train_dl = y_train[train_index].copy()
    y_train_dl = to_categorical(LE.fit_transform(y_train_dl))

    X_val_dl = X_train_val[val_index].copy()
    X_val_dl = np.transpose(X_val_dl, (0,2,1))
    y_val_dl = y_train[val_index].copy()
    y_val_dl = to_categorical(LE.transform(y_val_dl))

    X_test_dl = np.transpose(X_test, (0,2,1))
    y_test_dl = to_categorical(LE.transform(y_test))
    
    acc_cnn = dl_func('CNN',
                     X_train_dl, 
                     y_train_dl,
                     X_val_dl,
                     y_val_dl,
                     X_test_dl,
                     y_test_dl)
    
    acc_it = dl_func('InceptionTime',
                     X_train_dl, 
                     y_train_dl,
                     X_val_dl,
                     y_val_dl,
                     X_test_dl,
                     y_test_dl)

    accuracies_cnn.append(acc_cnn.copy())
    accuracies_it.append(acc_it.copy())

scores['dl_cnn'] = [np.mean(accuracies_cnn), np.std(accuracies_cnn)]
scores['dl_it'] = [np.mean(accuracies_it), np.std(accuracies_it)]

The value of model_types is set from ['CNN', 'DeepConvLSTM', 'ResNet', 'InceptionTime'] (default) to ['CNN']
The value of metrics is set from ['accuracy'] (default) to ['accuracy']
Train on 404 samples, validate on 100 samples
Epoch 1/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1372 - accuracy: 1.0000
Epoch 00001: val_accuracy improved from -inf to 0.95000, saving model to best_model.h5
404/404 [==============================] - 6s 15ms/sample - loss: 0.1381 - accuracy: 1.0000 - val_loss: 0.2747 - val_accuracy: 0.9500
Epoch 2/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1347 - accuracy: 1.0000
Epoch 00002: val_accuracy improved from 0.95000 to 0.98000, saving model to best_model.h5
404/404 [==============================] - 6s 16ms/sample - loss: 0.1381 - accuracy: 1.0000 - val_loss: 0.2680 - val_accuracy: 0.9800
Epoch 3/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1355 - accuracy: 1.0000
Epoch 00003: val_accuracy did not imp

Epoch 29/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1062 - accuracy: 1.0000
Epoch 00029: val_accuracy did not improve from 0.98000
404/404 [==============================] - 4s 9ms/sample - loss: 0.1078 - accuracy: 1.0000 - val_loss: 0.2526 - val_accuracy: 0.9700
Epoch 30/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1062 - accuracy: 1.0000
Epoch 00030: val_accuracy did not improve from 0.98000
404/404 [==============================] - 4s 9ms/sample - loss: 0.1068 - accuracy: 1.0000 - val_loss: 0.2764 - val_accuracy: 0.9400
Epoch 31/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1003 - accuracy: 1.0000
Epoch 00031: val_accuracy did not improve from 0.98000
404/404 [==============================] - 4s 9ms/sample - loss: 0.1022 - accuracy: 1.0000 - val_loss: 0.2523 - val_accuracy: 0.9600
Epoch 32/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0962 - accuracy: 1.0000
Epoch 00032: val_accuracy did not improve fr

Epoch 25/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0027 - accuracy: 1.0000
Epoch 00025: val_accuracy did not improve from 0.97000
404/404 [==============================] - 11s 27ms/sample - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.1703 - val_accuracy: 0.9500
Epoch 26/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0038 - accuracy: 1.0000
Epoch 00026: val_accuracy did not improve from 0.97000
404/404 [==============================] - 11s 27ms/sample - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.1664 - val_accuracy: 0.9400
Epoch 27/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0079 - accuracy: 1.0000
Epoch 00027: val_accuracy did not improve from 0.97000
404/404 [==============================] - 12s 29ms/sample - loss: 0.0085 - accuracy: 1.0000 - val_loss: 0.2205 - val_accuracy: 0.9300
Epoch 28/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0032 - accuracy: 1.0000
Epoch 00028: val_accuracy did not impr

Epoch 22/200
384/404 [===========================>..] - ETA: 0s - loss: 0.2069 - accuracy: 0.9974
Epoch 00022: val_accuracy did not improve from 0.99010
404/404 [==============================] - 2s 4ms/sample - loss: 0.2066 - accuracy: 0.9975 - val_loss: 0.5310 - val_accuracy: 0.8317
Epoch 23/200
384/404 [===========================>..] - ETA: 0s - loss: 0.2015 - accuracy: 0.9974
Epoch 00023: val_accuracy did not improve from 0.99010
404/404 [==============================] - 2s 4ms/sample - loss: 0.2023 - accuracy: 0.9975 - val_loss: 0.3251 - val_accuracy: 0.9802
Epoch 24/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1930 - accuracy: 1.0000
Epoch 00024: val_accuracy did not improve from 0.99010
404/404 [==============================] - 2s 5ms/sample - loss: 0.1926 - accuracy: 1.0000 - val_loss: 0.3101 - val_accuracy: 0.9505
Epoch 25/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1815 - accuracy: 1.0000
Epoch 00025: val_accuracy did not improve fr

Epoch 15/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0778 - accuracy: 0.9688
Epoch 00015: val_accuracy did not improve from 0.96040
404/404 [==============================] - 7s 17ms/sample - loss: 0.0772 - accuracy: 0.9678 - val_loss: 0.5629 - val_accuracy: 0.8119
Epoch 16/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0521 - accuracy: 0.9870
Epoch 00016: val_accuracy did not improve from 0.96040
404/404 [==============================] - 8s 21ms/sample - loss: 0.0518 - accuracy: 0.9876 - val_loss: 0.3461 - val_accuracy: 0.8812
Epoch 17/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0610 - accuracy: 0.9792
Epoch 00017: val_accuracy did not improve from 0.96040
404/404 [==============================] - 8s 19ms/sample - loss: 0.0697 - accuracy: 0.9752 - val_loss: 0.2165 - val_accuracy: 0.9109
Epoch 18/200
384/404 [===========================>..] - ETA: 0s - loss: 0.0707 - accuracy: 0.9740
Epoch 00018: val_accuracy did not improve

...

384/404 [===========================>..] - ETA: 0s - loss: 0.2892 - accuracy: 0.8802
Epoch 00038: val_accuracy did not improve from 0.75248
404/404 [==============================] - 10s 25ms/sample - loss: 0.2842 - accuracy: 0.8812 - val_loss: 2.1690 - val_accuracy: 0.5149
Epoch 39/200
384/404 [===========================>..] - ETA: 0s - loss: 0.2844 - accuracy: 0.8802
Epoch 00039: val_accuracy did not improve from 0.75248
404/404 [==============================] - 10s 25ms/sample - loss: 0.2847 - accuracy: 0.8787 - val_loss: 2.2235 - val_accuracy: 0.5743
Epoch 40/200
384/404 [===========================>..] - ETA: 0s - loss: 0.2412 - accuracy: 0.9193
Epoch 00040: val_accuracy did not improve from 0.75248
404/404 [==============================] - 10s 25ms/sample - loss: 0.2392 - accuracy: 0.9158 - val_loss: 1.5851 - val_accuracy: 0.6832
Epoch 41/200
384/404 [===========================>..] - ETA: 0s - loss: 0.2366 - accuracy: 0.8932
Epoch 00041: val_accuracy did not improve from 0.75

384/404 [===========================>..] - ETA: 0s - loss: 0.1461 - accuracy: 0.9583
Epoch 00066: val_accuracy did not improve from 0.90099
404/404 [==============================] - 10s 25ms/sample - loss: 0.1439 - accuracy: 0.9579 - val_loss: 0.5860 - val_accuracy: 0.8119
Epoch 67/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1220 - accuracy: 0.9583
Epoch 00067: val_accuracy did not improve from 0.90099
404/404 [==============================] - 10s 25ms/sample - loss: 0.1235 - accuracy: 0.9579 - val_loss: 0.3956 - val_accuracy: 0.8713
Epoch 68/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1052 - accuracy: 0.9609
Epoch 00068: val_accuracy did not improve from 0.90099
404/404 [==============================] - 10s 25ms/sample - loss: 0.1045 - accuracy: 0.9629 - val_loss: 0.5767 - val_accuracy: 0.8416
Epoch 69/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1318 - accuracy: 0.9531
Epoch 00069: val_accuracy did not improve from 0.90

Epoch 3/200
384/404 [===========================>..] - ETA: 0s - loss: 0.6286 - accuracy: 0.8672
Epoch 00003: val_accuracy did not improve from 0.95050
404/404 [==============================] - 3s 7ms/sample - loss: 0.6250 - accuracy: 0.8688 - val_loss: 1.0240 - val_accuracy: 0.4950
Epoch 4/200
384/404 [===========================>..] - ETA: 0s - loss: 0.5619 - accuracy: 0.8958
Epoch 00004: val_accuracy did not improve from 0.95050
404/404 [==============================] - 3s 7ms/sample - loss: 0.5653 - accuracy: 0.8960 - val_loss: 0.9994 - val_accuracy: 0.4950
Epoch 5/200
384/404 [===========================>..] - ETA: 0s - loss: 0.4992 - accuracy: 0.9167
Epoch 00005: val_accuracy did not improve from 0.95050
404/404 [==============================] - 3s 7ms/sample - loss: 0.4950 - accuracy: 0.9208 - val_loss: 1.1048 - val_accuracy: 0.4950
Epoch 6/200
384/404 [===========================>..] - ETA: 0s - loss: 0.4903 - accuracy: 0.9010
Epoch 00006: val_accuracy did not improve from 0

Epoch 00031: early stopping
The value of model_types is set from ['CNN', 'DeepConvLSTM', 'ResNet', 'InceptionTime'] (default) to ['InceptionTime']
The value of metrics is set from ['accuracy'] (default) to ['accuracy']
Train on 404 samples, validate on 101 samples
Epoch 1/200
384/404 [===========================>..] - ETA: 0s - loss: 0.2448 - accuracy: 0.8958
Epoch 00001: val_accuracy improved from -inf to 0.91089, saving model to best_model.h5
404/404 [==============================] - 5s 12ms/sample - loss: 0.2372 - accuracy: 0.9010 - val_loss: 0.2158 - val_accuracy: 0.9109
Epoch 2/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1996 - accuracy: 0.9193
Epoch 00002: val_accuracy did not improve from 0.91089
404/404 [==============================] - 5s 12ms/sample - loss: 0.2010 - accuracy: 0.9158 - val_loss: 0.8406 - val_accuracy: 0.7030
Epoch 3/200
384/404 [===========================>..] - ETA: 0s - loss: 0.2446 - accuracy: 0.8828
Epoch 00003: val_accuracy did not 

384/404 [===========================>..] - ETA: 0s - loss: 0.1801 - accuracy: 0.9245
Epoch 00028: val_accuracy did not improve from 0.95050
404/404 [==============================] - 5s 12ms/sample - loss: 0.1958 - accuracy: 0.9158 - val_loss: 0.3344 - val_accuracy: 0.8713
Epoch 29/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1984 - accuracy: 0.9271
Epoch 00029: val_accuracy did not improve from 0.95050
404/404 [==============================] - 5s 12ms/sample - loss: 0.1912 - accuracy: 0.9307 - val_loss: 0.2789 - val_accuracy: 0.8614
Epoch 30/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1938 - accuracy: 0.9141
Epoch 00030: val_accuracy did not improve from 0.95050
404/404 [==============================] - 5s 12ms/sample - loss: 0.1931 - accuracy: 0.9158 - val_loss: 0.1989 - val_accuracy: 0.9307
Epoch 31/200
384/404 [===========================>..] - ETA: 0s - loss: 0.1985 - accuracy: 0.9167
Epoch 00031: val_accuracy did not improve from 0.95050

### PyEEG

In [8]:
pyeeg_lr_acc = []
pyeeg_rf_acc = []
pyeeg_svc_acc = []
pyeeg_lgbm_acc = []

skf = StratifiedKFold(n_splits=10)
for train_index, test_index in tqdm(skf.split(X_full, y_full)):
    
    X_train = X_full[train_index]
    y_train = y_full[train_index]
    X_test = X_full[test_index]
    y_test = y_full[test_index]
    
    feat_dict = {}

    for df,name in zip([X_train,X_test],['X_train_pyeeg','X_test_pyeeg']):
        feat_dict[name] = np.array([]).reshape(-1,450)

        for ind in range(0,df.shape[0]):
            features = np.array([])

            for channel in range(0,df[ind].shape[0]):

                eeg_series = df[ind][channel]

                coefs = wavedec(eeg_series, 'db4', level=4)

                for band in coefs:
                    features = np.hstack([features,get_features(band)])

                fft_power = pyeeg.bin_power(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 256)
                power_spectral_intensity = fft_power[0]
                relative_intensity_ratio = fft_power[1]
#                 fisher_info = pyeeg.fisher_info(eeg_series, Tau = 1, DE = 90)
                pfd = pyeeg.pfd(eeg_series)
                spectral_entropy = pyeeg.spectral_entropy(eeg_series, Band = [0.5,4,7,12,30,100], Fs = 256,
                                                          Power_Ratio = relative_intensity_ratio)

#                 eeg_series = ResampleLinear1D(eeg_series,200)

#                 ap_entropy = pyeeg.ap_entropy(eeg_series, M = 3, R = 0.25*np.std(eeg_series))
                hfd = pyeeg.hfd(eeg_series, Kmax = 10)
                hjorth = pyeeg.hjorth(eeg_series)
                hjorth_mob = hjorth[0]
                hjorth_comp = hjorth[1]
    #             hurst = pyeeg.hurst(eeg_series)


                features = np.hstack([features, power_spectral_intensity, relative_intensity_ratio,
                                      hfd, hjorth_mob, hjorth_comp, pfd, spectral_entropy])


            feat_dict[name] = np.vstack([feat_dict[name],features])

    X_train_pyeeg = feat_dict['X_train_pyeeg'].copy()
    X_test_pyeeg = feat_dict['X_test_pyeeg'].copy()
    
    X_train_pyeeg = np.nan_to_num(X_train_pyeeg)
    X_test_pyeeg = np.nan_to_num(X_test_pyeeg)
    scaler = preprocessing.StandardScaler().fit(X_train_pyeeg)
    X_train_pyeeg = scaler.transform(X_train_pyeeg)
    X_test_pyeeg = scaler.transform(X_test_pyeeg)
    
    #pyeeg + lr    
    hyperparameters = {}
    hyperparameters['pyeeg_lr'] = {}
    hyperparameters['pyeeg_lr']['C'] = 1

    logistic = LogisticRegression(solver = 'lbfgs',
                                  multi_class = 'auto',
                                  max_iter=3000,
                                  C = hyperparameters['pyeeg_lr']['C'])

    logistic.fit(X_train_pyeeg,y_train)
    acc = logistic.score(X_test_pyeeg,y_test)
    pyeeg_lr_acc.append(np.round(acc,2).copy())
    
    #pyeeg + rf    
    hyperparameters = {}
    hyperparameters['pyeeg_rf'] = {}
    hyperparameters['pyeeg_rf']['n_estimators'] = 500
    hyperparameters['pyeeg_rf']['max_depth'] = 3

    rf = RandomForestClassifier(n_estimators = hyperparameters['pyeeg_rf']['n_estimators'],
                                max_depth = hyperparameters['pyeeg_rf']['max_depth'])

    rf.fit(X_train_pyeeg,y_train)
    acc = rf.score(X_test_pyeeg,y_test)
    pyeeg_rf_acc.append(np.round(acc,2).copy())
    
    #pyeeg + svc
    hyperparameters = {}
    hyperparameters['pyeeg_svc'] = {}
    hyperparameters['pyeeg_svc']['C'] = 1
    hyperparameters['pyeeg_svc']['kernel'] = 'rbf'
    hyperparameters['pyeeg_svc']['degree'] = 3
    hyperparameters['pyeeg_svc']['gamma'] = 'scale'

    svc = SVC(C = hyperparameters['pyeeg_svc']['C'],
              kernel = hyperparameters['pyeeg_svc']['kernel'],
              degree = hyperparameters['pyeeg_svc']['degree'],
              gamma = hyperparameters['pyeeg_svc']['gamma'])

    clf = make_pipeline(svc)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_svc_acc.append(np.round(acc,2).copy())

    
    #pyeeg + lgbm
    hyperparameters = {}
    hyperparameters['pyeeg_lgbm'] = {}
    hyperparameters['pyeeg_lgbm']['num_leaves'] = 254
    hyperparameters['pyeeg_lgbm']['max_depth'] = 2
    hyperparameters['pyeeg_lgbm']['learning_rate'] = 0.1
    hyperparameters['pyeeg_lgbm']['n_estimators'] = 500
    hyperparameters['pyeeg_lgbm']['min_split_gain'] = 0.2
    hyperparameters['pyeeg_lgbm']['min_child_samples'] = 20
    hyperparameters['pyeeg_lgbm']['colsample_bytree'] = 1
    hyperparameters['pyeeg_lgbm']['reg_alpha'] = 0.1
    hyperparameters['pyeeg_lgbm']['reg_lambda'] = 0

    def sparse_float(mat):
        return mat.astype('float')

    trans_sparse_float = FunctionTransformer(sparse_float, validate = False)

    lgbm = LGBMClassifier(n_jobs = -1,
                          num_leaves = hyperparameters['pyeeg_lgbm']['num_leaves'],
                          max_depth = hyperparameters['pyeeg_lgbm']['max_depth'],
                          learning_rate = hyperparameters['pyeeg_lgbm']['learning_rate'],
                          n_estimators = hyperparameters['pyeeg_lgbm']['n_estimators'],
                          min_split_gain = hyperparameters['pyeeg_lgbm']['min_split_gain'],
                          min_child_samples = hyperparameters['pyeeg_lgbm']['min_child_samples'],
                          colsample_by_tree = hyperparameters['pyeeg_lgbm']['colsample_bytree'],
                          reg_alpha = hyperparameters['pyeeg_lgbm']['reg_alpha'],
                          reg_lambda = hyperparameters['pyeeg_lgbm']['reg_lambda'])

    clf = make_pipeline(trans_sparse_float, lgbm)
    clf.fit(X_train_pyeeg,y_train)
    acc = clf.score(X_test_pyeeg,y_test)
    pyeeg_lgbm_acc.append(np.round(acc,2).copy())

scores['pyeeg_lr'] = [np.mean(pyeeg_lr_acc), np.std(pyeeg_lr_acc)]
scores['pyeeg_rf'] = [np.mean(pyeeg_rf_acc), np.std(pyeeg_rf_acc)]
scores['pyeeg_svc'] = [np.mean(pyeeg_svc_acc), np.std(pyeeg_svc_acc)]
scores['pyeeg_lgbm'] = [np.mean(pyeeg_lgbm_acc), np.std(pyeeg_lgbm_acc)]


/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pyeeg-0.4.4-py3.6.egg/pyeeg/fractal_dimension.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pyeeg-0.4.4-py3.6.egg/pyeeg/fractal_dimension.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/jet/prs/workspace/tf_2/lib/python3.6/site-packages/pyeeg-0.4.4-py3.6.egg/pyeeg/fractal_dimension.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are th

In [ ]:
#Salva scores
individual_results = pd.DataFrame({'dtw_acc' : dtw_acc,
                                   'wm_lr_acc' : accuracies_lr,
                                   'wm_rf_acc' : accuracies_rf,
                                   'wm_svc_acc' : accuracies_svc,
                                   'wm_lgbm_acc' : accuracies_lgbm,
                                   'cnn_acc' : accuracies_cnn,
                                   'it_acc' : accuracies_it,
                                   'pyeeg_lr_acc' : pyeeg_lr_acc,
                                   'pyeeg_rf_acc' : pyeeg_rf_acc,
                                   'pyeeg_svc_acc' : pyeeg_svc_acc,
                                   'pyeeg_lgbm_acc' : pyeeg_lgbm_acc})

individual_results.to_csv('results/Individual_Scores_{}.csv'.format(dataset), index = False)

results = pd.DataFrame(scores).T.reset_index().rename(columns={0 : 'mean',
                                                               1 : 'std',
                                                               'index' : 'method'}).sort_values('mean', ascending = False)

results.to_csv('results/Scores_{}.csv'.format(dataset), index = False)

In [ ]:
results

,method,mean,std
9,pyeeg_svc,0.859000,0.069058
10,pyeeg_lgbm,0.852000,0.045782
7,pyeeg_lr,0.850000,0.054589
6,dl_it,0.841385,0.070652
8,pyeeg_rf,0.835000,0.049850
5,dl_cnn,0.834336,0.056599
0,dtw_knn,0.830702,0.088200
3,wm_svc,0.826000,0.046519
2,wm_rf,0.824000,0.055893
4,wm_lgbm,0.799000,0.072450
